## 임베딩

이전 예제에서는 `vocab_size` 길이의 고차원 bag-of-words 벡터를 사용했고, 저차원 위치 표현 벡터를 희소한 원-핫 표현으로 명시적으로 변환했습니다. 하지만 이 원-핫 표현은 메모리 효율적이지 않습니다. 게다가 각 단어가 서로 독립적으로 처리되기 때문에 원-핫 인코딩된 벡터는 단어 간의 의미적 유사성을 표현하지 못합니다.

이번 단원에서는 **News AG** 데이터셋을 계속 탐구할 것입니다. 시작하기 위해 데이터를 로드하고 이전 단원에서 정의를 가져오겠습니다.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### 임베딩이란 무엇인가?

**임베딩**의 아이디어는 단어를 낮은 차원의 밀집 벡터로 표현하여 단어의 의미론적 의미를 반영하는 것입니다. 나중에 의미 있는 단어 임베딩을 구축하는 방법에 대해 논의하겠지만, 지금은 임베딩을 단어 벡터의 차원을 줄이는 방법으로 생각해봅시다.

임베딩 레이어는 단어를 입력으로 받아 지정된 `embedding_size`의 출력 벡터를 생성합니다. 어느 정도로는 `Dense` 레이어와 매우 유사하지만, 원-핫 인코딩된 벡터를 입력으로 받는 대신 단어 번호를 입력으로 받을 수 있습니다.

네트워크의 첫 번째 레이어로 임베딩 레이어를 사용하면, bag-of-words 모델에서 **embedding bag** 모델로 전환할 수 있습니다. 여기서 텍스트의 각 단어를 해당 임베딩으로 변환한 후, `sum`, `average`, `max`와 같은 집계 함수를 사용하여 모든 임베딩을 계산합니다.

![다섯 개의 시퀀스 단어에 대한 임베딩 분류기를 보여주는 이미지.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

우리의 분류기 신경망은 다음 레이어들로 구성됩니다:

* `TextVectorization` 레이어: 문자열을 입력으로 받아 토큰 번호의 텐서를 생성합니다. 적절한 어휘 크기 `vocab_size`를 지정하고, 자주 사용되지 않는 단어는 무시합니다. 입력 형태는 1이고, 출력 형태는 $n$입니다. 결과적으로 $n$개의 토큰을 얻으며, 각 토큰은 0에서 `vocab_size` 사이의 숫자를 포함합니다.
* `Embedding` 레이어: $n$개의 숫자를 받아 각 숫자를 주어진 길이의 밀집 벡터로 줄입니다 (예: 100). 따라서 $n$ 형태의 입력 텐서는 $n\times 100$ 형태의 텐서로 변환됩니다.
* 집계 레이어: 첫 번째 축을 따라 이 텐서의 평균을 계산합니다. 즉, 서로 다른 단어에 해당하는 모든 $n$ 입력 텐서의 평균을 계산합니다. 이 레이어를 구현하기 위해 `Lambda` 레이어를 사용하고 평균을 계산하는 함수를 전달합니다. 출력 형태는 100이며, 전체 입력 시퀀스의 수치적 표현이 됩니다.
* 최종 `Dense` 선형 분류기.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` 출력에서 **output shape** 열의 첫 번째 텐서 차원 `None`은 미니배치 크기를 나타내며, 두 번째 차원은 토큰 시퀀스의 길이를 나타냅니다. 미니배치 내 모든 토큰 시퀀스는 서로 다른 길이를 가지고 있습니다. 다음 섹션에서 이를 처리하는 방법에 대해 논의하겠습니다.

이제 네트워크를 훈련시켜 봅시다:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **참고** 우리는 데이터의 일부를 기반으로 벡터라이저를 구축하고 있습니다. 이는 프로세스를 가속화하기 위해 수행되며, 이로 인해 텍스트의 모든 토큰이 어휘에 포함되지 않을 수 있습니다. 이 경우 해당 토큰은 무시되며 약간 낮은 정확도를 초래할 수 있습니다. 그러나 실제로 텍스트의 일부는 종종 좋은 어휘 추정을 제공합니다.


### 변수 시퀀스 크기 처리하기

미니배치에서 훈련이 어떻게 이루어지는지 이해해 봅시다. 위 예시에서 입력 텐서는 차원이 1이고, 128개의 미니배치를 사용하므로 텐서의 실제 크기는 $128 \times 1$입니다. 하지만 각 문장에 포함된 토큰 수는 서로 다릅니다. `TextVectorization` 레이어를 단일 입력에 적용하면, 텍스트가 어떻게 토큰화되었는지에 따라 반환되는 토큰 수가 달라집니다:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


그러나 여러 시퀀스에 벡터라이저를 적용할 때 직사각형 모양의 텐서를 생성해야 하므로 사용되지 않은 요소를 PAD 토큰(우리의 경우 0)으로 채웁니다.


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

여기에서 임베딩을 볼 수 있습니다:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **참고**: 패딩 양을 최소화하기 위해, 경우에 따라 데이터셋의 모든 시퀀스를 길이가 증가하는 순서(더 정확히는 토큰 수)에 따라 정렬하는 것이 합리적일 수 있습니다. 이렇게 하면 각 미니배치가 유사한 길이의 시퀀스를 포함하도록 할 수 있습니다.


## 시맨틱 임베딩: Word2Vec

이전 예제에서 임베딩 레이어는 단어를 벡터 표현으로 매핑하는 방법을 학습했지만, 이러한 표현은 의미론적 의미를 가지지 않았습니다. 비슷한 단어나 동의어가 어떤 벡터 거리(예: 유클리드 거리)를 기준으로 서로 가까운 벡터에 해당하도록 벡터 표현을 학습할 수 있다면 좋을 것입니다.

이를 위해, [Word2Vec](https://en.wikipedia.org/wiki/Word2vec)과 같은 기법을 사용하여 대규모 텍스트 컬렉션에서 임베딩 모델을 사전 학습해야 합니다. Word2Vec은 단어의 분산 표현을 생성하는 데 사용되는 두 가지 주요 아키텍처를 기반으로 합니다:

 - **Continuous bag-of-words** (CBoW): 주변 문맥에서 단어를 예측하도록 모델을 학습합니다. n그램 $(W_{-2},W_{-1},W_0,W_1,W_2)$가 주어졌을 때, 모델의 목표는 $(W_{-2},W_{-1},W_1,W_2)$로부터 $W_0$를 예측하는 것입니다.
 - **Continuous skip-gram**: CBoW와 반대입니다. 이 모델은 현재 단어를 예측하기 위해 주변 문맥 단어의 윈도우를 사용합니다.

CBoW는 속도가 빠르며, skip-gram은 더 느리지만 드문 단어를 표현하는 데 더 효과적입니다.

![단어를 벡터로 변환하는 CBoW와 Skip-Gram 알고리즘을 보여주는 이미지.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Google News 데이터셋에서 사전 학습된 Word2Vec 임베딩을 실험하려면 **gensim** 라이브러리를 사용할 수 있습니다. 아래는 'neural'과 가장 유사한 단어를 찾는 예제입니다.

> **참고:** 처음으로 단어 벡터를 생성할 때, 다운로드에 시간이 걸릴 수 있습니다!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


단어에서 벡터 임베딩을 추출하여 분류 모델 훈련에 사용할 수 있습니다. 임베딩은 300개의 구성 요소를 가지지만, 명확성을 위해 여기서는 벡터의 첫 20개 구성 요소만 표시합니다:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

의미 임베딩의 훌륭한 점은 의미를 기반으로 벡터 인코딩을 조작할 수 있다는 것입니다. 예를 들어, *king*과 *woman*의 벡터 표현에 최대한 가깝고 *man*이라는 단어에서 최대한 멀리 떨어진 단어를 찾을 수 있습니다:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

위의 예는 일부 내부 GenSym 마법을 사용하지만, 기본 논리는 실제로 매우 간단합니다. 임베딩에 대한 흥미로운 점은 임베딩 벡터에서 일반적인 벡터 연산을 수행할 수 있으며, 이는 단어 **의미**에 대한 연산을 반영한다는 것입니다. 위의 예는 벡터 연산으로 표현될 수 있습니다: 우리는 **KING-MAN+WOMAN**에 해당하는 벡터를 계산하고(해당 단어의 벡터 표현에서 `+`와 `-` 연산을 수행), 그런 다음 그 벡터에 가장 가까운 단어를 사전에서 찾습니다:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: 우리는 *man*과 *woman* 벡터에 작은 계수를 추가해야 했습니다. 이를 제거하고 어떤 일이 발생하는지 확인해 보세요.

가장 가까운 벡터를 찾기 위해, 우리는 TensorFlow의 메커니즘을 사용하여 우리의 벡터와 어휘에 있는 모든 벡터 간의 거리 벡터를 계산한 다음, `argmin`을 사용하여 최소 단어의 인덱스를 찾습니다.


Word2Vec은 단어의 의미를 표현하는 훌륭한 방법처럼 보이지만, 다음과 같은 여러 단점이 있습니다:

* CBoW와 skip-gram 모델은 **예측 임베딩**으로, 로컬 컨텍스트만 고려합니다. Word2Vec은 글로벌 컨텍스트를 활용하지 않습니다.
* Word2Vec은 단어의 **형태론**을 고려하지 않습니다. 즉, 단어의 의미가 단어의 다른 부분(예: 어근)에 따라 달라질 수 있다는 점을 반영하지 않습니다.

**FastText**는 두 번째 한계를 극복하려고 시도하며, Word2Vec을 기반으로 각 단어와 단어 내에서 발견되는 문자 n-그램에 대한 벡터 표현을 학습합니다. 그런 다음, 각 학습 단계에서 이러한 표현 값을 하나의 벡터로 평균화합니다. 이는 사전 학습에 많은 추가 계산을 요구하지만, 단어 임베딩이 서브워드 정보를 인코딩할 수 있도록 합니다.

또 다른 방법인 **GloVe**는 단어 임베딩에 대해 다른 접근 방식을 사용하며, 단어-컨텍스트 행렬의 분해를 기반으로 합니다. 먼저, 다양한 컨텍스트에서 단어가 등장하는 횟수를 세는 큰 행렬을 생성한 후, 이 행렬을 낮은 차원으로 표현하여 재구성 손실을 최소화하려고 합니다.

gensim 라이브러리는 이러한 단어 임베딩을 지원하며, 위의 모델 로딩 코드를 변경하여 이를 실험해볼 수 있습니다.


## Keras에서 사전 학습된 임베딩 사용하기

위의 예제를 수정하여 임베딩 레이어의 행렬을 Word2Vec과 같은 의미론적 임베딩으로 미리 채울 수 있습니다. 사전 학습된 임베딩의 어휘와 텍스트 코퍼스의 어휘는 일치하지 않을 가능성이 높으므로 하나를 선택해야 합니다. 여기서는 두 가지 가능한 옵션을 탐구합니다: 토크나이저 어휘를 사용하는 방법과 Word2Vec 임베딩의 어휘를 사용하는 방법.

### 토크나이저 어휘 사용하기

토크나이저 어휘를 사용할 때, 어휘의 일부 단어는 Word2Vec 임베딩과 대응되지만 일부는 누락될 수 있습니다. 우리의 어휘 크기가 `vocab_size`이고 Word2Vec 임베딩 벡터 길이가 `embed_size`일 때, 임베딩 레이어는 `vocab_size`$\times$`embed_size` 형태의 가중치 행렬로 표현됩니다. 우리는 어휘를 순회하며 이 행렬을 채울 것입니다:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Word2Vec 어휘에 없는 단어들은 0으로 남겨두거나, 랜덤 벡터를 생성할 수 있습니다.

이제 사전 학습된 가중치를 사용하여 임베딩 레이어를 정의할 수 있습니다:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Note**: `Embedding`을 생성할 때 `trainable=False`로 설정한 것을 주목하세요. 이는 Embedding 레이어를 재학습하지 않는다는 것을 의미합니다. 이로 인해 정확도가 약간 낮아질 수 있지만, 학습 속도는 빨라집니다.

### 임베딩 어휘 사용하기

이전 접근 방식의 문제점 중 하나는 TextVectorization과 Embedding에서 사용되는 어휘가 서로 다르다는 점입니다. 이 문제를 해결하기 위해 다음과 같은 방법을 사용할 수 있습니다:
* Word2Vec 모델을 우리의 어휘로 재학습합니다.
* 사전 학습된 Word2Vec 모델의 어휘를 사용하여 우리의 데이터셋을 로드합니다. 데이터셋을 로드할 때 사용되는 어휘는 로드 과정에서 지정할 수 있습니다.

후자의 방법이 더 간단해 보이므로 이를 구현해 보겠습니다. 먼저, Word2Vec 임베딩에서 가져온 지정된 어휘를 사용하여 `TextVectorization` 레이어를 생성하겠습니다:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

gensim 워드 임베딩 라이브러리에는 `get_keras_embeddings`라는 편리한 함수가 포함되어 있으며, 이를 사용하면 자동으로 해당 Keras 임베딩 레이어를 생성할 수 있습니다.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


우리 데이터셋의 일부 단어가 사전 학습된 GloVe 어휘에 없기 때문에 더 높은 정확도를 보지 못하는 이유 중 하나입니다. 따라서 이러한 단어들은 본질적으로 무시됩니다. 이를 극복하기 위해 우리는 데이터셋을 기반으로 자체 임베딩을 학습할 수 있습니다.


## 문맥적 임베딩

Word2Vec와 같은 전통적인 사전 학습 임베딩 표현의 주요 한계 중 하나는 단어의 일부 의미를 포착할 수는 있지만, 서로 다른 의미를 구별하지 못한다는 점입니다. 이는 후속 모델에서 문제를 일으킬 수 있습니다.

예를 들어, 'play'라는 단어는 다음 두 문장에서 서로 다른 의미를 가집니다:
- 나는 극장에서 **연극**을 보았다.
- 존은 친구들과 **놀고** 싶어 한다.

우리가 언급한 사전 학습 임베딩은 'play'라는 단어의 두 가지 의미를 동일한 임베딩으로 표현합니다. 이러한 한계를 극복하기 위해, 우리는 **언어 모델**을 기반으로 한 임베딩을 구축해야 합니다. 언어 모델은 방대한 텍스트 코퍼스에서 학습되며, 단어들이 서로 다른 문맥에서 어떻게 조합될 수 있는지를 *이해*합니다. 문맥적 임베딩에 대한 논의는 이 튜토리얼의 범위를 벗어나지만, 다음 단원에서 언어 모델을 다룰 때 다시 논의할 것입니다.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전이 권위 있는 출처로 간주되어야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
